In [1]:
import numpy as np

In [2]:
def loadExData():
    return [[0,0,1,0,0,2,0,0,0,0,5],
           [0,0,0,5,0,3,0,0,0,0,3],
           [0,0,0,0,4,1,0,1,0,4,0],
           [3,3,4,0,0,0,0,2,2,0,0],
           [5,4,2,0,0,0,0,5,5,0,0],
           [0,0,0,0,5,0,1,0,0,0,0],
           [4,1,4,0,0,0,0,4,5,0,1],
           [0,0,0,4,0,4,0,0,0,0,4],
           [0,0,0,2,0,2,5,0,0,1,2],
           [1,0,0,4,0,0,0,1,2,0,0]]

In [3]:
myMat = np.mat(loadExData())
U,Sigma,VT = np.linalg.svd(myMat)
j = 0
for i in Sigma:
    Sigma[j] = round(i,3)
    j += 1
print(Sigma)
print(VT.shape)

[14.27  11.198  7.135  5.136  4.686  3.097  2.729  2.556  1.058  0.185]
(11, 11)


In [4]:
print(sum(Sigma**2))
print(sum(Sigma[0:3]**2))
print(round(sum(Sigma[0:3]**2)/sum(Sigma**2),2))
NewData = np.round(U[:, :4] * np.mat(np.eye(4)*Sigma[:4])*VT[:4,:],3)
MyData = np.round(U*np.mat(np.eye(10) * Sigma)*VT[:10,:],3)
#np.eye(4)是4阶的对角矩阵
#print(np.mat(np.eye(4)))
print(NewData)
print('\n')
print(NewData)

452.99899600000003
379.936329
0.84
[[ 0.067  0.156  1.344 -0.055 -0.309  2.492  0.817 -0.067 -0.286  0.077
   4.417]
 [ 0.007 -0.208 -0.309  4.832 -0.267  2.919  0.876  0.041  0.329  0.087
   2.984]
 [ 0.118  0.096  0.034  0.035  4.611  0.659  1.196  0.683  0.109  2.396
   0.029]
 [ 2.983  1.953  2.538 -0.544 -0.082 -0.03  -0.045  2.777  3.003 -0.014
   0.571]
 [ 4.812  3.048  3.459  0.395  0.123 -0.327 -0.144  4.568  5.077  0.068
  -0.185]
 [-0.095 -0.04  -0.154 -0.152  3.878  0.358  0.942  0.392 -0.109  1.997
  -0.258]
 [ 4.186  2.7    3.415 -0.019 -0.06   0.229  0.029  3.921  4.289  0.026
   0.928]
 [-0.052 -0.174  0.148  4.098 -0.253  3.383  1.059 -0.056  0.094  0.144
   4.1  ]
 [-0.105 -0.128  0.124  2.243  1.123  2.264  0.99   0.045 -0.064  0.755
   2.644]
 [ 1.123  0.48  -0.204  3.839 -0.129  0.661  0.117  1.153  1.645 -0.034
  -0.382]]


[[ 0.067  0.156  1.344 -0.055 -0.309  2.492  0.817 -0.067 -0.286  0.077
   4.417]
 [ 0.007 -0.208 -0.309  4.832 -0.267  2.919  0.876  0.041  0

In [5]:
print(Sigma)
a = []
j = 1
for i in Sigma:
    a.append(i)
a.append(0)
print(a)
print(type(a))
a = np.array(a)
print(type(a))

[14.27  11.198  7.135  5.136  4.686  3.097  2.729  2.556  1.058  0.185]
[14.27, 11.198, 7.135, 5.136, 4.686, 3.097, 2.729, 2.556, 1.058, 0.185, 0]
<class 'list'>
<class 'numpy.ndarray'>


In [6]:
#基于SVD的评分估计
#dataMat是输入矩阵
#simMeas是相似度计算函数
#user和item是待打分的用户和物品
def svdEst(userData,xformedItems, user, simMeas, item) :
    n = xformedItems.shape[0]
    simTotal = 0.0; ratSimTotal = 0.0
    # 对于给定的用户，for循环所有物品，计算与item的相似度
    for j in range(n) :
        userRating = userData[:,j]
        if userRating == 0 or j == item : continue
        similarity = simMeas(xformedItems[item, :].T, xformedItems[j, :].T)
        # print便于了解相似度计算的进展情况
        print ('the %d and %d similarity is : %f' % (item, j, similarity))
        # 对相似度求和
        simTotal += similarity
        # 对相似度及评分值的乘积求和
        ratSimTotal += similarity * userRating
    if simTotal == 0 : return 0
    else : return ratSimTotal/simTotal

In [7]:
def cosSim(a,b):
    c = (a.T * b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return c

def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=svdEst):
    U,Sigma,VT = np.linalg.svd(dataMat)
    # 使用奇异值构建一个对角矩阵
    Sig4 = np.mat(np.eye(4)*Sigma[:4])
    # 利用U矩阵将物品转换到低维空间中
    xformedItems = dataMat.T * U[:, :4] * Sig4.I
    print('xformedItems=',xformedItems)
    print('xformedItems行和列数', xformedItems.shape)
    
    unratedItems = np.nonzero(dataMat[user, :].A == 0)[1]
    print('dataMat[user, :].A=',dataMat[user, :].A)
    print('nonzero(dataMat[user, :].A == 0)结果为', np.nonzero(dataMat[user, :].A == 0))
    # 如果不存在未评分物品，退出函数，否则在所有未评分物品上进行循环
    if len(unratedItems) == 0: return ('you rated everything')
    itemScores = []
    for item in unratedItems:
        print('item=',item)
   # 对于每个未评分物品，通过调用standEst()来产生该物品基于相似度的预测评分。
        estimatedScore = estMethod(dataMat[user, :],xformedItems, user, simMeas, item)
        # 该物品的编号和估计得分值会放在一个元素列表itemScores
        itemScores.append((item, estimatedScore))
    # 寻找前N个未评级物品
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

In [8]:
myMat= np.mat(loadExData())
result=recommend(myMat, 1, estMethod=svdEst)
#print(result)

xformedItems= [[-4.94993155e-01 -8.55622175e-02 -2.48611806e-02  3.33301232e-02]
 [-3.13458547e-01 -7.52514984e-02 -7.48135856e-03 -3.50541410e-02]
 [-3.78450772e-01 -4.30110266e-02 -2.73248525e-02 -3.16127753e-01]
 [-9.66651163e-02  5.96631453e-01 -1.11181970e-01  7.08108827e-01]
 [-1.48606103e-02  4.20778007e-02  8.58513789e-01  4.28827005e-02]
 [-5.34500615e-02  4.98695994e-01  4.12721285e-02 -1.51444295e-01]
 [-1.53015570e-02  1.73637176e-01  1.96700179e-01 -5.96048540e-02]
 [-4.69111042e-01 -7.14523528e-02  8.17335025e-02  6.74284466e-02]
 [-5.18848182e-01 -6.64352384e-02 -3.46135811e-02  1.62309665e-01]
 [-1.56328052e-02  6.08803417e-02  4.39273573e-01  4.45686018e-04]
 [-1.06449559e-01  5.79097298e-01 -8.90946777e-02 -5.80592096e-01]]
xformedItems行和列数 (11, 4)
dataMat[user, :].A= [[0 0 0 5 0 3 0 0 0 0 3]]
nonzero(dataMat[user, :].A == 0)结果为 (array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int64), array([0, 1, 2, 4, 6, 7, 8, 9], dtype=int64))
item= 0
the 0 and 3 similarity is : 0.049016
the